# General Experiments

### Scope
In this notebook, we use one of the possible configurations of our LightGCNPlus model to experiment with different hyperparameters and configurations.

### Purpose
The purpose is to showcase why we fixed certain hyperparameters and configurations in the hyperparameter tuning loop in the LightGCNPlus.ipynb notebook.

### Results
Given the report results from the training we chose to fix the following hyperparameters and configurations:
- ...

In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from train import train_model
from models import LightGCNPlus
from config import DEVICE
from train import train_model
from postprocess import report_training_results
from itertools import product

In [2]:
from load import load_train_data
train_df = load_train_data()

In [3]:
from preprocess import preprocess

# old: A_tilde, \
# standardized_train_ratings, \
# train_users, train_items, \
# means, stds, \
# val_users, \
# val_items, \
# orig_val_ratings, \
# standardized_val_ratings \

# new (but unformatted): A_tilde, train_users, train_items, train_ratings, val_users, val_items, val_ratings

# new formatted

A_tilde, \
train_users, train_items, train_ratings, \
val_users, val_items, val_ratings \
= preprocess(train_df)

In [4]:
# Model and optimizer hyperparameters
INIT_EMBS_STD=0.075
LR=0.1
WEIGHT_DECAY=0.00005
DROPOUT=0.5
ACT_FN = nn.GELU()

# Train loop hyperparameters
EPOCHS = 4000
STOP_THRESHOLD=1e-09

# To be searched (example values)
# K=28
# L=4
# PROJECTIONS = (4,)

In [5]:
# Grid
ks = [28]  # different Ks work with different projections
layers = [5]  # tested 3 already
projections = [(6,)]

num_combinations = len(ks) * len(layers) * len(projections)
print(f"Searching over {num_combinations} combinations.")

Searching over 1 combinations.


In [6]:
# Tuning LightGCNPlus
results = {
    "min_val_losses": [],
    "params": []
}
for K in ks:
    for L in layers:
        for C in projections:
            model = LightGCNPlus(A_tilde, ACT_FN, K, L, INIT_EMBS_STD, DROPOUT, C).to(DEVICE)
            optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
            loss_fn = nn.MSELoss()
            print(f"K={K}, L={L}, C={C}")
            train_rmse, val_rmse = train_model(model, optimizer, loss_fn, train_users, train_items, train_ratings, val_users, val_items, val_ratings, EPOCHS, STOP_THRESHOLD, save_best_model=True, verbosity=1)
            report_training_results(train_rmse, val_rmse)
            
            results["min_val_losses"].append(min(val_rmse))
            results["params"].append((K, L, C))

K=28, L=5, C=(6,)
Epoch 1 - Avg loss in last 1 epochs: - Train: 3.7978 - Val: 1.3642 - Best val: 1.3642 at epoch 2
Epoch 2 - Avg loss in last 1 epochs: - Train: 1.4846 - Val: 9.1415 - Best val: 1.3642 at epoch 2
Epoch 3 - Avg loss in last 1 epochs: - Train: 9.2527 - Val: 1.5486 - Best val: 1.3642 at epoch 2
Epoch 4 - Avg loss in last 1 epochs: - Train: 1.6803 - Val: 2.5276 - Best val: 1.3642 at epoch 2
Epoch 5 - Avg loss in last 1 epochs: - Train: 2.5495 - Val: 2.8019 - Best val: 1.3642 at epoch 2
Epoch 6 - Avg loss in last 1 epochs: - Train: 2.8179 - Val: 2.7773 - Best val: 1.3642 at epoch 2
Epoch 7 - Avg loss in last 1 epochs: - Train: 2.8071 - Val: 2.6616 - Best val: 1.3642 at epoch 2
Epoch 8 - Avg loss in last 1 epochs: - Train: 2.7240 - Val: 2.1516 - Best val: 1.3642 at epoch 2
Epoch 9 - Avg loss in last 1 epochs: - Train: 2.3485 - Val: 2.2270 - Best val: 1.3642 at epoch 2
Epoch 10 - Avg loss in last 1 epochs: - Train: 2.6168 - Val: 2.5992 - Best val: 1.3642 at epoch 2
Epoch 11 - 

In [ ]:
# Report top k best hyperparameter combos
TOP_K = 10
best_ids = np.argsort(results["min_val_losses"])[:TOP_K]

for i in best_ids:
    print(f"Best hyperparameters: {results['params'][i]}")
    print(f"Best val loss: {results['min_val_losses'][i]}")

In [ ]:
from postprocess import load_submission_users_items, report_submission_results, create_submission_matrix, reverse_standardize, report_clip_data, to_submission_format, load_means_stds, load_best_val_model

# Read model that achieved best validation loss
submission_users, submission_items = load_submission_users_items()
# Load model inputs
# model = load_best_val_model(model_class, ID)
# Get predictions for submission

raw_pred_ratings = model.get_ratings(submission_users, submission_items).detach().cpu().numpy()
report_submission_results(raw_pred_ratings, "raw")
raw_submission_matrix = create_submission_matrix(raw_pred_ratings, submission_users, submission_items)
means, stds = load_means_stds(ID)
submission_matrix = reverse_standardize(raw_submission_matrix, means, stds)
pred_ratings = submission_matrix[submission_users, submission_items]

report_clip_data(pred_ratings)
pred_ratings = np.clip(pred_ratings, 1, 5)

report_submission_results(pred_ratings, "clipped")

submission = to_submission_format(submission_users, submission_items, pred_ratings)
submission.to_csv('../data/submission_data/submission.csv', index=False)